# Level 1: Basic Cross-Validation

## 🎯 Learning Objectives

By the end of this exercise, you will be able to:
1. **Understand** why we need cross-validation
2. **Implement** basic k-fold cross-validation
3. **Interpret** cross-validation results
4. **Compare** different cross-validation strategies

## ⏱️ Time Estimate
**30-45 minutes** (take your time to understand each concept)

## 📚 Prerequisites
- Basic Python programming
- Understanding of train/test splits
- Familiarity with scikit-learn basics

---

## 🤔 Why Do We Need Cross-Validation?

### The Problem with Single Train/Test Split

**Real-world analogy**: Imagine judging a student's performance based on just one exam. What if:
- The student had a bad day?
- The exam was unusually easy or hard?
- The topics didn't represent the full curriculum?

**Same problem with ML models**: A single train/test split might:
- Give overly optimistic or pessimistic results
- Not represent the true model performance
- Be influenced by random chance in data splitting

### The Cross-Validation Solution

**Cross-validation** is like giving multiple exams and averaging the scores:
- More reliable performance estimate
- Reduces impact of random variation
- Better understanding of model stability

## 🛠️ Setup: Import Libraries and Create Data

**What we're doing**: Setting up our environment and creating a practice dataset.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning tools
from sklearn.datasets import make_classification
from sklearn.model_selection import (
    train_test_split, cross_val_score, KFold, StratifiedKFold
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Set random seed for reproducible results
np.random.seed(42)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("🎯 Ready to learn cross-validation!")

### Create Practice Dataset

**What we're doing**: Creating a synthetic classification dataset to practice with.

**Why synthetic data**: 
- We know the "ground truth"
- Controlled complexity
- Focus on learning concepts, not data cleaning

In [ ]:
# Create a synthetic classification dataset
print("🔧 Creating practice dataset...")

X, y = make_classification(
    n_samples=1000,        # 1000 data points
    n_features=10,         # 10 input features
    n_informative=8,       # 8 features are actually useful
    n_redundant=2,         # 2 features are combinations of others
    n_clusters_per_class=1, # Simple structure
    random_state=42        # For reproducible results
)

print(f"📊 Dataset created:")
print(f"   • {X.shape[0]} samples (data points)")
print(f"   • {X.shape[1]} features (input variables)")
print(f"   • Target distribution: {np.bincount(y)}")
print(f"     - Class 0: {np.bincount(y)[0]} samples")
print(f"     - Class 1: {np.bincount(y)[1]} samples")

# Scale the features (important for many algorithms)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n⚖️ Features scaled to have mean=0 and std=1")
print(f"   • Mean: {X_scaled.mean():.3f}")
print(f"   • Standard deviation: {X_scaled.std():.3f}")

---

## 📖 Part 1: Understanding the Problem with Single Split

**What we're doing**: Demonstrating why a single train/test split can be unreliable.

**The experiment**: 
1. Perform multiple random train/test splits
2. Train the same model on each split
3. See how much the results vary

In [ ]:
print("🔬 Experiment: Multiple Random Train/Test Splits")
print("=" * 50)

# We'll try 10 different random splits
n_splits = 10
scores = []

# Create our model (Logistic Regression)
model = LogisticRegression(random_state=42, max_iter=1000)

print("Performing 10 different train/test splits...\n")

for i in range(n_splits):
    # Different random_state for each split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=i
    )
    
    # Train and evaluate
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    scores.append(score)
    
    print(f"Split {i+1:2d}: Accuracy = {score:.4f}")

# Analyze the results
scores = np.array(scores)
print("\n📊 Results Summary:")
print(f"   • Mean accuracy: {scores.mean():.4f}")
print(f"   • Standard deviation: {scores.std():.4f}")
print(f"   • Range: {scores.min():.4f} to {scores.max():.4f}")
print(f"   • Difference: {scores.max() - scores.min():.4f}")

# Visualize the variation
plt.figure(figsize=(10, 6))
plt.plot(range(1, n_splits + 1), scores, 'bo-', linewidth=2, markersize=8)
plt.axhline(y=scores.mean(), color='red', linestyle='--', 
            label=f'Mean: {scores.mean():.4f}')
plt.fill_between(range(1, n_splits + 1), 
                 scores.mean() - scores.std(), 
                 scores.mean() + scores.std(), 
                 alpha=0.2, color='red', 
                 label=f'±1 std: {scores.std():.4f}')

plt.xlabel('Split Number')
plt.ylabel('Accuracy')
plt.title('Accuracy Variation Across Different Train/Test Splits')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🤔 What this shows:")
if scores.std() > 0.02:
    print("   ⚠️ High variation! Single split results are unreliable.")
else:
    print("   ✅ Low variation, but cross-validation is still better practice.")
    
print("   💡 This is why we need cross-validation for reliable estimates!")

### 🎯 Checkpoint Question 1

**Think about it**: Why do you think the accuracy scores vary across different train/test splits?

<details>
<summary>Click for explanation</summary>

**Answer**: The variation occurs because:
1. **Random sampling**: Different splits include different subsets of data
2. **Data distribution**: Some splits might have easier/harder test samples
3. **Class balance**: Random splits might not preserve class proportions perfectly
4. **Sample size**: With limited data, small changes in samples can affect results

This variation is exactly why cross-validation gives us more reliable estimates!

</details>

---

## 📖 Part 2: Basic K-Fold Cross-Validation

**What is K-Fold Cross-Validation?**

**The process**:
1. **Split** data into K equal parts ("folds")
2. **For each fold**:
   - Use that fold as test set
   - Use remaining K-1 folds as training set
   - Train model and evaluate
3. **Average** the K results

**Visual representation** (5-fold example):
```
Fold 1: [TEST] [TRAIN] [TRAIN] [TRAIN] [TRAIN]
Fold 2: [TRAIN] [TEST] [TRAIN] [TRAIN] [TRAIN]
Fold 3: [TRAIN] [TRAIN] [TEST] [TRAIN] [TRAIN]
Fold 4: [TRAIN] [TRAIN] [TRAIN] [TEST] [TRAIN]
Fold 5: [TRAIN] [TRAIN] [TRAIN] [TRAIN] [TEST]
```

### Manual Implementation (Understanding the Process)

**What we're doing**: Implementing k-fold CV step-by-step to understand the process.

In [ ]:
print("🔧 Manual 5-Fold Cross-Validation Implementation")
print("=" * 50)

# Set up 5-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Store results for each fold
fold_scores = []
fold_details = []

print(f"Splitting {len(X_scaled)} samples into {k_folds} folds...\n")

# Perform cross-validation manually
for fold_num, (train_idx, test_idx) in enumerate(kf.split(X_scaled), 1):
    print(f"📁 Fold {fold_num}:")
    
    # Split data for this fold
    X_train_fold = X_scaled[train_idx]
    X_test_fold = X_scaled[test_idx]
    y_train_fold = y[train_idx]
    y_test_fold = y[test_idx]
    
    print(f"   • Training samples: {len(X_train_fold)}")
    print(f"   • Test samples: {len(X_test_fold)}")
    
    # Train model
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train_fold, y_train_fold)
    
    # Evaluate
    score = model.score(X_test_fold, y_test_fold)
    fold_scores.append(score)
    
    # Store details
    fold_details.append({
        'fold': fold_num,
        'train_size': len(X_train_fold),
        'test_size': len(X_test_fold),
        'accuracy': score
    })
    
    print(f"   • Accuracy: {score:.4f}")
    print()

# Calculate final results
cv_mean = np.mean(fold_scores)
cv_std = np.std(fold_scores)

print("📊 Cross-Validation Results:")
print(f"   • Individual fold scores: {[f'{score:.4f}' for score in fold_scores]}")
print(f"   • Mean CV score: {cv_mean:.4f}")
print(f"   • Standard deviation: {cv_std:.4f}")
print(f"   • 95% Confidence interval: {cv_mean:.4f} ± {cv_std * 2:.4f}")

### Using Scikit-Learn's Built-in Function

**What we're doing**: Using scikit-learn's `cross_val_score` function for the same task.

**Why this is better**: 
- Less code
- Less error-prone
- Optimized implementation

In [ ]:
print("🚀 Using Scikit-Learn's cross_val_score")
print("=" * 40)

# Create model
model = LogisticRegression(random_state=42, max_iter=1000)

# Perform 5-fold cross-validation
cv_scores_sklearn = cross_val_score(
    model, X_scaled, y, cv=5, scoring='accuracy'
)

print(f"Cross-validation scores: {[f'{score:.4f}' for score in cv_scores_sklearn]}")
print(f"Mean: {cv_scores_sklearn.mean():.4f}")
print(f"Std:  {cv_scores_sklearn.std():.4f}")

# Compare with our manual implementation
print("\n🔍 Comparison with Manual Implementation:")
print(f"Manual mean:    {cv_mean:.4f}")
print(f"Sklearn mean:   {cv_scores_sklearn.mean():.4f}")
print(f"Difference:     {abs(cv_mean - cv_scores_sklearn.mean()):.6f}")

if abs(cv_mean - cv_scores_sklearn.mean()) < 0.001:
    print("✅ Results match! Our manual implementation is correct.")
else:
    print("⚠️ Small differences due to different random splits.")

### 🎯 Checkpoint Question 2

**Your turn!** Complete the code below to perform 10-fold cross-validation:

```python
# TODO: Perform 10-fold cross-validation
cv_scores_10fold = cross_val_score(
    model, X_scaled, y, cv=___, scoring='accuracy'  # Fill in the blank
)

print(f"10-fold CV mean: {cv_scores_10fold.mean():.4f}")
print(f"10-fold CV std:  {cv_scores_10fold.std():.4f}")
```

In [ ]:
# TODO: Complete this code
cv_scores_10fold = cross_val_score(
    model, X_scaled, y, cv=___, scoring='accuracy'  # Replace ___ with the correct value
)

print(f"10-fold CV mean: {cv_scores_10fold.mean():.4f}")
print(f"10-fold CV std:  {cv_scores_10fold.std():.4f}")

# Compare with 5-fold
print(f"\n📊 Comparison:")
print(f"5-fold:  {cv_scores_sklearn.mean():.4f} ± {cv_scores_sklearn.std():.4f}")
print(f"10-fold: {cv_scores_10fold.mean():.4f} ± {cv_scores_10fold.std():.4f}")

<details>
<summary>Click for solution</summary>

```python
cv_scores_10fold = cross_val_score(
    model, X_scaled, y, cv=10, scoring='accuracy'
)
```

**Explanation**: The `cv` parameter specifies the number of folds. For 10-fold cross-validation, we use `cv=10`.

</details>

---

## 📖 Part 3: Stratified Cross-Validation

**What is Stratified Cross-Validation?**

**The problem**: Regular k-fold might create imbalanced folds by chance.

**The solution**: Stratified k-fold ensures each fold has the same class distribution as the original dataset.

**When to use**: 
- **Always** for classification problems
- Especially important with imbalanced datasets

### Comparing Regular vs Stratified K-Fold

**What we're doing**: Creating an imbalanced dataset and comparing the two approaches.

In [ ]:
print("🔬 Experiment: Regular vs Stratified K-Fold")
print("=" * 45)

# Create an imbalanced dataset
X_imb, y_imb = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=8,
    weights=[0.8, 0.2],  # 80% class 0, 20% class 1
    random_state=42
)

X_imb_scaled = StandardScaler().fit_transform(X_imb)

print(f"📊 Imbalanced dataset created:")
print(f"   • Total samples: {len(y_imb)}")
print(f"   • Class 0: {np.sum(y_imb == 0)} ({np.sum(y_imb == 0)/len(y_imb)*100:.1f}%)")
print(f"   • Class 1: {np.sum(y_imb == 1)} ({np.sum(y_imb == 1)/len(y_imb)*100:.1f}%)")

# Function to check class distribution in folds
def analyze_fold_distribution(cv_method, X, y, method_name):
    print(f"\n📁 {method_name} - Fold Analysis:")
    
    fold_distributions = []
    for fold_num, (train_idx, test_idx) in enumerate(cv_method.split(X, y), 1):
        test_y = y[test_idx]
        class_0_pct = np.sum(test_y == 0) / len(test_y) * 100
        class_1_pct = np.sum(test_y == 1) / len(test_y) * 100
        
        fold_distributions.append((class_0_pct, class_1_pct))
        print(f"   Fold {fold_num}: Class 0: {class_0_pct:.1f}%, Class 1: {class_1_pct:.1f}%")
    
    return fold_distributions

# Regular K-Fold
regular_kf = KFold(n_splits=5, shuffle=True, random_state=42)
regular_dist = analyze_fold_distribution(regular_kf, X_imb_scaled, y_imb, "Regular K-Fold")

# Stratified K-Fold
stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
stratified_dist = analyze_fold_distribution(stratified_kf, X_imb_scaled, y_imb, "Stratified K-Fold")

# Calculate variation in class distributions
regular_class1_pcts = [dist[1] for dist in regular_dist]
stratified_class1_pcts = [dist[1] for dist in stratified_dist]

print(f"\n📊 Class 1 Distribution Variation:")
print(f"   Regular K-Fold:    {np.std(regular_class1_pcts):.2f}% std deviation")
print(f"   Stratified K-Fold: {np.std(stratified_class1_pcts):.2f}% std deviation")

if np.std(regular_class1_pcts) > np.std(stratified_class1_pcts):
    print("   ✅ Stratified K-Fold has more consistent class distributions!")
else:
    print("   ⚠️ In this case, both methods are similar.")

### Performance Comparison

**What we're doing**: Comparing model performance using both CV methods.

In [ ]:
print("⚖️ Performance Comparison: Regular vs Stratified CV")
print("=" * 50)

model = LogisticRegression(random_state=42, max_iter=1000)

# Regular K-Fold CV
regular_scores = cross_val_score(
    model, X_imb_scaled, y_imb, cv=KFold(n_splits=5, shuffle=True, random_state=42)
)

# Stratified K-Fold CV
stratified_scores = cross_val_score(
    model, X_imb_scaled, y_imb, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
)

print(f"📊 Results:")
print(f"Regular K-Fold:")
print(f"   • Scores: {[f'{score:.4f}' for score in regular_scores]}")
print(f"   • Mean: {regular_scores.mean():.4f} ± {regular_scores.std():.4f}")

print(f"\nStratified K-Fold:")
print(f"   • Scores: {[f'{score:.4f}' for score in stratified_scores]}")
print(f"   • Mean: {stratified_scores.mean():.4f} ± {stratified_scores.std():.4f}")

# Visualize the comparison
plt.figure(figsize=(12, 6))

# Plot scores
x_pos = np.arange(5)
width = 0.35

plt.bar(x_pos - width/2, regular_scores, width, label='Regular K-Fold', alpha=0.8)
plt.bar(x_pos + width/2, stratified_scores, width, label='Stratified K-Fold', alpha=0.8)

plt.xlabel('Fold Number')
plt.ylabel('Accuracy')
plt.title('Cross-Validation Scores: Regular vs Stratified K-Fold')
plt.xticks(x_pos, [f'Fold {i+1}' for i in range(5)])
plt.legend()
plt.grid(True, alpha=0.3)

# Add mean lines
plt.axhline(y=regular_scores.mean(), color='blue', linestyle='--', alpha=0.7)
plt.axhline(y=stratified_scores.mean(), color='orange', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

print(f"\n🔍 Analysis:")
if stratified_scores.std() < regular_scores.std():
    print("   ✅ Stratified CV shows more consistent results!")
else:
    print("   ⚠️ Both methods show similar consistency in this case.")

print(f"\n💡 Key Takeaway: Always use Stratified K-Fold for classification!")

### 🎯 Checkpoint Question 3

**Your turn!** Why is Stratified K-Fold better for imbalanced datasets?

<details>
<summary>Click for explanation</summary>

**Answer**: Stratified K-Fold is better because:
1. **Preserves class distribution**: Each fold has the same proportion of classes as the original dataset
2. **Reduces variance**: More consistent performance estimates across folds
3. **Prevents bias**: Avoids folds with very few (or no) minority class samples
4. **Better representation**: Each fold is a better representative sample of the overall data

This is especially important when classes are imbalanced!

</details>

---

## 📖 Part 4: Choosing the Right Number of Folds

**The question**: How many folds should you use?

**Common choices**:
- **5-fold**: Good balance of bias and variance, computationally efficient
- **10-fold**: Lower bias, higher variance, more computation
- **Leave-One-Out (LOO)**: Lowest bias, highest variance, very expensive

**Trade-offs**:
- **More folds**: Lower bias (closer to true performance), higher variance, more computation
- **Fewer folds**: Higher bias, lower variance, less computation

In [ ]:
print("🔬 Experiment: Comparing Different Numbers of Folds")
print("=" * 50)

model = LogisticRegression(random_state=42, max_iter=1000)
fold_options = [3, 5, 10, 20]
results = {}

for k in fold_options:
    print(f"\n📊 {k}-Fold Cross-Validation:")
    
    # Perform k-fold CV
    scores = cross_val_score(model, X_scaled, y, cv=k, scoring='accuracy')
    
    results[k] = {
        'scores': scores,
        'mean': scores.mean(),
        'std': scores.std(),
        'training_size_per_fold': len(X_scaled) * (k-1) / k
    }
    
    print(f"   • Mean accuracy: {scores.mean():.4f}")
    print(f"   • Std deviation: {scores.std():.4f}")
    print(f"   • Training size per fold: {results[k]['training_size_per_fold']:.0f} samples")

# Visualize the comparison
plt.figure(figsize=(14, 10))

# Plot 1: Mean accuracy and error bars
plt.subplot(2, 2, 1)
means = [results[k]['mean'] for k in fold_options]
stds = [results[k]['std'] for k in fold_options]

plt.errorbar(fold_options, means, yerr=stds, marker='o', capsize=5, capthick=2, linewidth=2)
plt.xlabel('Number of Folds')
plt.ylabel('Mean Accuracy')
plt.title('Mean Accuracy vs Number of Folds')
plt.grid(True, alpha=0.3)

# Plot 2: Standard deviation
plt.subplot(2, 2, 2)
plt.plot(fold_options, stds, 'ro-', linewidth=2, markersize=8)
plt.xlabel('Number of Folds')
plt.ylabel('Standard Deviation')
plt.title('Variance vs Number of Folds')
plt.grid(True, alpha=0.3)

# Plot 3: Training set size
plt.subplot(2, 2, 3)
training_sizes = [results[k]['training_size_per_fold'] for k in fold_options]
plt.plot(fold_options, training_sizes, 'go-', linewidth=2, markersize=8)
plt.xlabel('Number of Folds')
plt.ylabel('Training Samples per Fold')
plt.title('Training Set Size vs Number of Folds')
plt.grid(True, alpha=0.3)

# Plot 4: All scores distribution
plt.subplot(2, 2, 4)
for i, k in enumerate(fold_options):
    scores = results[k]['scores']
    plt.scatter([k] * len(scores), scores, alpha=0.7, s=50, label=f'{k}-fold')

plt.xlabel('Number of Folds')
plt.ylabel('Individual Fold Scores')
plt.title('Score Distribution by Number of Folds')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Summary:")
for k in fold_options:
    print(f"   {k}-fold: {results[k]['mean']:.4f} ± {results[k]['std']:.4f}")

print(f"\n💡 Recommendations:")
print(f"   • For most cases: Use 5-fold or 10-fold CV")
print(f"   • Small datasets: Use 10-fold or Leave-One-Out")
print(f"   • Large datasets: 3-fold or 5-fold may be sufficient")
print(f"   • Always consider computational cost vs. accuracy trade-off")

---

## 🎯 Final Exercise: Put It All Together

**Your challenge**: Complete the following cross-validation analysis!

**Task**: Compare two different models using proper cross-validation.

In [ ]:
print("🎯 Final Challenge: Model Comparison with Cross-Validation")
print("=" * 55)

# TODO: Complete this analysis
# 1. Create two models: LogisticRegression and RandomForestClassifier
model1 = LogisticRegression(random_state=42, max_iter=1000)
model2 = ___  # TODO: Create a RandomForestClassifier

# 2. Use 5-fold stratified cross-validation for both models
cv_strategy = ___  # TODO: Create StratifiedKFold with 5 splits

# 3. Get cross-validation scores for both models
scores1 = cross_val_score(model1, X_scaled, y, cv=cv_strategy, scoring='accuracy')
scores2 = ___  # TODO: Get scores for model2

# 4. Compare the results
print(f"Logistic Regression:")
print(f"   • Scores: {[f'{score:.4f}' for score in scores1]}")
print(f"   • Mean: {scores1.mean():.4f} ± {scores1.std():.4f}")

print(f"\nRandom Forest:")
print(f"   • Scores: {[f'{score:.4f}' for score in scores2]}")
print(f"   • Mean: {scores2.mean():.4f} ± {scores2.std():.4f}")

# 5. Statistical comparison
from scipy import stats
t_stat, p_value = stats.ttest_rel(scores1, scores2)

print(f"\n📊 Statistical Comparison:")
print(f"   • Difference in means: {scores2.mean() - scores1.mean():.4f}")
print(f"   • T-statistic: {t_stat:.4f}")
print(f"   • P-value: {p_value:.4f}")

if p_value < 0.05:
    if scores2.mean() > scores1.mean():
        print(f"   ✅ Random Forest is significantly better!")
    else:
        print(f"   ✅ Logistic Regression is significantly better!")
else:
    print(f"   ⚖️ No significant difference between models.")

# 6. Visualize the comparison
plt.figure(figsize=(10, 6))
x_pos = np.arange(5)
width = 0.35

plt.bar(x_pos - width/2, scores1, width, label='Logistic Regression', alpha=0.8)
plt.bar(x_pos + width/2, scores2, width, label='Random Forest', alpha=0.8)

plt.xlabel('Fold Number')
plt.ylabel('Accuracy')
plt.title('Model Comparison: Cross-Validation Scores')
plt.xticks(x_pos, [f'Fold {i+1}' for i in range(5)])
plt.legend()
plt.grid(True, alpha=0.3)

# Add mean lines
plt.axhline(y=scores1.mean(), color='blue', linestyle='--', alpha=0.7)
plt.axhline(y=scores2.mean(), color='orange', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

<details>
<summary>Click for solution</summary>

```python
# Complete solution:
model2 = RandomForestClassifier(random_state=42, n_estimators=100)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores2 = cross_val_score(model2, X_scaled, y, cv=cv_strategy, scoring='accuracy')
```

**Explanation**: 
- RandomForestClassifier is an ensemble method that often performs well
- StratifiedKFold ensures balanced class distribution in each fold
- We use the same CV strategy for fair comparison

</details>

---

## 🎉 Congratulations!

You've completed the **Basic Cross-Validation** exercise! Here's what you've learned:

### ✅ Key Concepts Mastered:
1. **Why cross-validation is essential** for reliable model evaluation
2. **How to implement k-fold cross-validation** both manually and with scikit-learn
3. **The importance of stratified cross-validation** for classification problems
4. **How to choose the right number of folds** based on your dataset and constraints
5. **How to compare models** using cross-validation results

### 🚀 Next Steps:

**Ready for more?** Choose your next exercise based on your comfort level:

#### Level 2 (Intermediate):
- [Complete Model Evaluation Pipeline](./level2_complete_evaluation_pipeline.ipynb)
- [Advanced Hyperparameter Optimization](./level2_advanced_hyperparameter_optimization.ipynb)

#### Level 3 (Advanced):
- [Healthcare Diagnostic Model](./level3_healthcare_diagnostic.ipynb)
- [Financial Risk Assessment](./level3_financial_risk_assessment.ipynb)

#### Additional Resources:
- **[Cross-Validation Guide](../cross-validation.md)**: Deep dive into advanced CV techniques
- **[Hyperparameter Tuning Guide](../hyperparameter-tuning.md)**: Learn optimization strategies
- **[Main Tutorial](../tutorial.ipynb)**: Comprehensive walkthrough of all concepts

### 💡 Remember:
- **Always use cross-validation** for reliable performance estimates
- **Use stratified CV** for classification problems
- **Consider computational cost** when choosing number of folds
- **Compare models fairly** using the same CV strategy

**Great job!** You're well on your way to mastering model evaluation! 🌟